In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import logging
from datetime import datetime
import numpy as np
import os

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('riwi_loader.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# CONFIGURACIÓN DE BASE DE DATOS - CAMBIA ESTOS VALORES
DB_CONFIG = {
    'database': 'riwi_analytics',
    'user': 'postgres',
    'password': 'Qwe.123*',  # ⚠️ CAMBIA ESTO
    'host': 'localhost',
    'port': 5432
}

ARCHIVO_CSV_LIMPIO = "riwi_datos_limpios.csv"  

In [3]:
class PostgreSQLLoaderFinal:
    def __init__(self, db_config, csv_file):
        self.db_config = db_config
        self.csv_file = csv_file
        self.estadisticas = {}
        self.engine = None
        self.conn = None
        
    def conectar_postgresql(self):
        """Establecer conexión directa con psycopg2"""
        try:
            self.conn = psycopg2.connect(**self.db_config)
            self.conn.autocommit = False
            logger.info("✅ Conexión psycopg2 establecida")
            return True
        except Exception as e:
            logger.error(f"❌ Error conexión psycopg2: {e}")
            return False
    
    def crear_engine_sqlalchemy(self):
        """Crear engine de SQLAlchemy para operaciones bulk"""
        try:
            self.engine = create_engine(
                f"postgresql://{self.db_config['user']}:{self.db_config['password']}@"
                f"{self.db_config['host']}:{self.db_config['port']}/{self.db_config['database']}",
                pool_pre_ping=True,
                echo=False
            )
            logger.info("✅ Engine SQLAlchemy creado")
            return True
        except Exception as e:
            logger.error(f"❌ Error creando engine: {e}")
            return False
    
    def validar_csv(self):
        """Validar que el archivo CSV limpio existe y tiene datos"""
        try:
            if not os.path.exists(self.csv_file):
                logger.error(f"❌ Archivo no encontrado: {self.csv_file}")
                return False
            
            # Verificar tamaño
            tamano_mb = os.path.getsize(self.csv_file) / (1024 * 1024)
            logger.info(f"📊 Tamaño CSV: {tamano_mb:.2f} MB")
            
            # Leer primeras filas para validar
            df_sample = pd.read_csv(self.csv_file, nrows=100)
            logger.info(f"✅ CSV válido encontrado: {self.csv_file}")
            logger.info(f"📋 Columnas: {list(df_sample.columns)}")
            
            # Contar total de filas
            with open(self.csv_file, 'r', encoding='utf-8') as f:
                total_filas = sum(1 for line in f) - 1  # Restar header
            
            logger.info(f"📈 Total filas: {total_filas:,}")
            self.estadisticas['total_filas_csv'] = total_filas
            
            return True
            
        except Exception as e:
            logger.error(f"❌ Error validando CSV: {e}")
            return False
    
    def cargar_csv(self):
        """Cargar el CSV limpio con manejo de tipos de datos"""
        try:
            logger.info("📖 Cargando CSV limpio...")
            
            # Leer CSV especificando tipos de datos para evitar errores
            dtype_dict = {
                'cantidad': 'float64',
                'precio_unitario': 'float64',
                'descuento': 'float64',
                'costo_envio': 'float64',
                'total': 'float64',
                'anio': 'int64',
                'mes': 'int64',
                'trimestre': 'int64'
            }
            
            df = pd.read_csv(self.csv_file, dtype=dtype_dict)
            
            # Asegurar que las columnas de texto sean strings y manejar NaN
            columnas_texto = ['producto', 'tipo_producto', 'ciudad', 'pais', 'tipo_venta', 'tipo_cliente']
            for col in columnas_texto:
                if col in df.columns:
                    df[col] = df[col].astype(str)
                    # Reemplazar 'nan' strings con valores vacíos
                    df[col] = df[col].replace('nan', '')
                    df[col] = df[col].str.strip()
            
            # Convertir fecha
            if 'fecha' in df.columns:
                df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
            
            logger.info(f"✅ CSV cargado: {len(df)} registros, {len(df.columns)} columnas")
            return df
            
        except Exception as e:
            logger.error(f"❌ Error cargando CSV: {e}")
            return None
    
    def validar_columnas_tabla(self):
        """Verificar que las columnas de la tabla existen"""
        try:
            query = """
                SELECT column_name 
                FROM information_schema.columns 
                WHERE table_schema = 'analytics' 
                AND table_name = 'ventas'
                ORDER BY ordinal_position
            """
            columnas_existentes = pd.read_sql(query, self.engine)['column_name'].tolist()
            logger.info(f"📋 Columnas existentes en tabla ventas: {columnas_existentes}")
            
            return columnas_existentes
            
        except Exception as e:
            logger.error(f"❌ Error verificando columnas: {e}")
            return []
    
    def cargar_dimensiones_basicas(self):
        """Cargar datos básicos de dimensiones"""
        try:
            logger.info("📊 Cargando dimensiones básicas...")
            
            # Datos predefinidos para dimensiones
            dimensiones_data = {
                'tipos_producto': [
                    (1, 'Hogar'), (2, 'Bebida'), (3, 'Lácteo'), 
                    (4, 'Alimento_Perecedero'), (5, 'Snacks'), (6, 'Abarrotes')
                ],
                'tipos_cliente': [
                    (1, 'Gobierno'), (2, 'Corporativo'), (3, 'Minorista'), (4, 'Mayorista')
                ],
                'canales_venta': [
                    (1, 'Tienda Fisica'), (2, 'Distribuidor'), (3, 'Call Center'), (4, 'Online')
                ]
            }
            
            with self.conn.cursor() as cursor:
                # Cargar tipos de producto
                for tipo_id, nombre in dimensiones_data['tipos_producto']:
                    cursor.execute("""
                        INSERT INTO analytics.tipos_producto (tipo_producto_id, nombre_tipo, categoria_principal)
                        VALUES (%s, %s, %s)
                        ON CONFLICT DO NOTHING
                    """, (tipo_id, nombre, 'Categoría'))
                
                # Cargar tipos de cliente
                for cliente_id, nombre in dimensiones_data['tipos_cliente']:
                    cursor.execute("""
                        INSERT INTO analytics.tipos_cliente (tipo_cliente_id, nombre_tipo, descripcion)
                        VALUES (%s, %s, %s)
                        ON CONFLICT DO NOTHING
                    """, (cliente_id, nombre, f"Cliente {nombre}"))
                
                # Cargar canales de venta
                for canal_id, nombre in dimensiones_data['canales_venta']:
                    es_digital = nombre in ['Call Center', 'Online']
                    cursor.execute("""
                        INSERT INTO analytics.canales_venta (canal_id, nombre_canal, tipo_canal, es_digital)
                        VALUES (%s, %s, %s, %s)
                        ON CONFLICT DO NOTHING
                    """, (canal_id, nombre, 'Principal', es_digital))
            
            self.conn.commit()
            logger.info("✅ Dimensiones básicas cargadas")
            return True
            
        except Exception as e:
            logger.error(f"❌ Error cargando dimensiones básicas: {e}")
            self.conn.rollback()
            return False
    
    def procesar_y_cargar(self, df):
        """Procesar dataframe y cargar a PostgreSQL con mapeo correcto de columnas"""
        try:
            logger.info("🔄 Procesando datos para carga...")
            
            # 1. CARGAR REGIONES ÚNICAS
            logger.info("🌍 Cargando regiones...")
            regiones_unicas = df[['pais', 'ciudad']].drop_duplicates().reset_index(drop=True)
            regiones_unicas['region_id'] = range(1, len(regiones_unicas) + 1)
            
            # Limpiar datos de región
            regiones_unicas['pais'] = regiones_unicas['pais'].astype(str).str.strip().str.upper()
            regiones_unicas['ciudad'] = regiones_unicas['ciudad'].astype(str).str.strip().str.upper()
            
            # Cargar regiones
            regiones_data = regiones_unicas[['region_id', 'pais', 'ciudad']].values.tolist()
            with self.conn.cursor() as cursor:
                for region_id, pais, ciudad in regiones_data:
                    # Manejar valores None o vacíos
                    if not pais or pais == 'NAN':
                        pais = 'SIN_PAIS'
                    if not ciudad or ciudad == 'NAN':
                        ciudad = 'SIN_CIUDAD'
                    
                    cursor.execute("""
                        INSERT INTO analytics.regiones (region_id, pais, ciudad, activo, fecha_creacion)
                        VALUES (%s, %s, %s, %s, %s)
                        ON CONFLICT DO NOTHING
                    """, (region_id, pais, ciudad, True, datetime.now()))
            self.conn.commit()
            logger.info(f"✅ {len(regiones_unicas)} regiones cargadas")
            
            # 2. OBTENER MAPA DE REGIONES
            regiones_map = pd.read_sql("SELECT region_id, pais, ciudad FROM analytics.regiones", self.engine)
            df = df.merge(regiones_map, on=['pais', 'ciudad'], how='left')
            
            # 3. CARGAR PRODUCTOS ÚNICOS
            logger.info("📦 Cargando productos...")
            productos_unicos = df[['producto', 'tipo_producto']].drop_duplicates().reset_index(drop=True)
            productos_unicos['producto_id'] = range(1, len(productos_unicos) + 1)
            
            # Limpiar datos de producto
            productos_unicos['producto'] = productos_unicos['producto'].astype(str).str.strip()
            productos_unicos['tipo_producto'] = productos_unicos['tipo_producto'].astype(str).str.strip()
            
            # Cargar productos
            productos_data = productos_unicos[['producto_id', 'producto', 'tipo_producto']].values.tolist()
            with self.conn.cursor() as cursor:
                for producto_id, nombre, tipo in productos_data:
                    # Manejar valores None o vacíos
                    if not nombre or nombre == 'nan':
                        nombre = 'SIN_NOMBRE'
                    if not tipo or tipo == 'nan':
                        tipo = 'SIN_TIPO'
                    
                    cursor.execute("""
                        INSERT INTO analytics.productos (producto_id, nombre_producto, categoria_sistema, activo, fecha_creacion)
                        VALUES (%s, %s, %s, %s, %s)
                        ON CONFLICT DO NOTHING
                    """, (producto_id, nombre.title(), tipo.title(), True, datetime.now()))
            self.conn.commit()
            logger.info(f"✅ {len(productos_unicos)} productos cargados")
            
            # 4. OBTENER MAPA DE PRODUCTOS
            productos_map = pd.read_sql("SELECT producto_id, nombre_producto FROM analytics.productos", self.engine)
            df = df.merge(productos_map, left_on='producto', right_on='nombre_producto', how='left')
            
            # 5. PREPARAR VENTAS CON MAPEO CORRECTO DE COLUMNAS
            logger.info("💰 Preparando ventas...")
            
            # Mapear tipos de cliente y venta a IDs
            cliente_map = {
                'Gobierno': 1, 'Corporativo': 2, 'Minorista': 3, 'Mayorista': 4,
                'Tienda Fisica': 1, 'Distribuidor': 2, 'Call Center': 3, 'Online': 4
            }
            
            # Preparar dataframe de ventas con nombres de columnas exactos de la BD
            ventas_df = pd.DataFrame()
            ventas_df['fecha_venta'] = pd.to_datetime(df['fecha'], errors='coerce')
            ventas_df['producto_id'] = pd.to_numeric(df['producto_id'], errors='coerce').fillna(0).astype(int)
            ventas_df['region_id'] = pd.to_numeric(df['region_id'], errors='coerce').fillna(0).astype(int)
            
            # Mapear tipos con manejo de valores faltantes
            ventas_df['canal_id'] = df['tipo_venta'].map(cliente_map).fillna(1).astype(int)
            ventas_df['tipo_cliente_id'] = df['tipo_cliente'].map(cliente_map).fillna(1).astype(int)
            
            # Columnas numéricas con nombres exactos de la BD
            ventas_df['cantidad'] = pd.to_numeric(df['cantidad'], errors='coerce').fillna(0)
            ventas_df['precio_unitario'] = pd.to_numeric(df['precio_unitario'], errors='coerce').fillna(0)
            ventas_df['descuento_porcentaje'] = pd.to_numeric(df['descuento'], errors='coerce').fillna(0)
            ventas_df['costo_envio'] = pd.to_numeric(df['costo_envio'], errors='coerce').fillna(0)
            ventas_df['monto_total'] = pd.to_numeric(df['total'], errors='coerce').fillna(0)
            ventas_df['fecha_carga'] = datetime.now()
            
            # Eliminar filas con datos críticos faltantes
            ventas_df = ventas_df.dropna(subset=['fecha_venta', 'producto_id', 'region_id'])
            ventas_df = ventas_df[(ventas_df['producto_id'] > 0) & (ventas_df['region_id'] > 0)]
            
            # 6. CARGAR VENTAS EN CHUNKS
            logger.info("📤 Cargando ventas...")
            chunk_size = 5000
            total_chunks = (len(ventas_df) // chunk_size) + 1
            
            for i in range(0, len(ventas_df), chunk_size):
                chunk = ventas_df.iloc[i:i+chunk_size]
                
                # Cargar chunk con columnas exactas
                chunk.to_sql(
                    'ventas', self.engine, schema='analytics',
                    if_exists='append', index=False, method='multi',
                    chunksize=1000
                )
                
                logger.info(f"📈 Chunk {i//chunk_size + 1}/{total_chunks}: {len(chunk)} registros")
            
            self.estadisticas['total_ventas'] = len(ventas_df)
            logger.info(f"✅ {len(ventas_df)} ventas cargadas")
            
            return True
            
        except Exception as e:
            logger.error(f"❌ Error en proceso de carga: {e}")
            import traceback
            logger.error(f"Traceback: {traceback.format_exc()}")
            self.conn.rollback()
            return False
    
    def crear_indices_optimizacion(self):
        """Crear índices para optimizar consultas"""
        try:
            logger.info("🔑 Creando índices de optimización...")
            
            queries_indice = [
                "CREATE INDEX IF NOT EXISTS idx_ventas_fecha ON analytics.ventas(fecha_venta)",
                "CREATE INDEX IF NOT EXISTS idx_ventas_producto ON analytics.ventas(producto_id)",
                "CREATE INDEX IF NOT EXISTS idx_ventas_region ON analytics.ventas(region_id)",
                "CREATE INDEX IF NOT EXISTS idx_ventas_canal ON analytics.ventas(canal_id)",
                "CREATE INDEX IF NOT EXISTS idx_ventas_cliente ON analytics.ventas(tipo_cliente_id)",
                "CREATE INDEX IF NOT EXISTS idx_productos_nombre ON analytics.productos(nombre_producto)",
                "CREATE INDEX IF NOT EXISTS idx_regiones_pais_ciudad ON analytics.regiones(pais, ciudad)"
            ]
            
            with self.conn.cursor() as cursor:
                for query in queries_indice:
                    cursor.execute(query)
            
            self.conn.commit()
            logger.info("✅ Índices creados exitosamente")
            return True
            
        except Exception as e:
            logger.error(f"❌ Error creando índices: {e}")
            return False
    
    def verificar_carga(self):
        """Verificar que los datos se cargaron correctamente"""
        try:
            logger.info("🔍 Verificando carga de datos...")
            
            queries_verificacion = {
                'total_ventas': "SELECT COUNT(*) as total FROM analytics.ventas",
                'total_productos': "SELECT COUNT(*) as total FROM analytics.productos",
                'total_regiones': "SELECT COUNT(*) as total FROM analytics.regiones",
                'rango_fechas': """
                    SELECT 
                        MIN(fecha_venta) as fecha_min,
                        MAX(fecha_venta) as fecha_max
                    FROM analytics.ventas
                """,
                'ventas_por_pais': """
                    SELECT pais, COUNT(*) as total_ventas
                    FROM analytics.ventas v
                    JOIN analytics.regiones r ON v.region_id = r.region_id
                    GROUP BY pais
                    ORDER BY total_ventas DESC
                    LIMIT 5
                """
            }
            
            resultados = {}
            for nombre, query in queries_verificacion.items():
                resultado = pd.read_sql(query, self.engine)
                resultados[nombre] = resultado
            
            # Mostrar resultados
            logger.info("📊 VERIFICACIÓN DE CARGA:")
            logger.info(f"  📈 Total ventas: {resultados['total_ventas'].iloc[0]['total']:,}")
            logger.info(f"  📦 Total productos: {resultados['total_productos'].iloc[0]['total']:,}")
            logger.info(f"  🌍 Total regiones: {resultados['total_regiones'].iloc[0]['total']:,}")
            
            fecha_min = resultados['rango_fechas'].iloc[0]['fecha_min']
            fecha_max = resultados['rango_fechas'].iloc[0]['fecha_max']
            logger.info(f"  📅 Rango fechas: {fecha_min} hasta {fecha_max}")
            
            logger.info("  🌎 Top 5 países:")
            for _, row in resultados['ventas_por_pais'].iterrows():
                logger.info(f"    {row['pais']}: {row['total_ventas']:,} ventas")
            
            # Guardar estadísticas
            self.estadisticas.update({
                'ventas_finales': int(resultados['total_ventas'].iloc[0]['total']),
                'productos_finales': int(resultados['total_productos'].iloc[0]['total']),
                'regiones_finales': int(resultados['total_regiones'].iloc[0]['total']),
                'fecha_min': str(fecha_min),
                'fecha_max': str(fecha_max)
            })
            
            return True
            
        except Exception as e:
            logger.error(f"❌ Error en verificación: {e}")
            return False
    
    def guardar_estadisticas(self):
        """Guardar estadísticas del proceso"""
        try:
            estadisticas_finales = {
                'timestamp': datetime.now().isoformat(),
                'archivo_csv': self.csv_file,
                'estadisticas': self.estadisticas,
                'base_datos': self.db_config['database']
            }
            
            with open('riwi_carga_estadisticas.json', 'w', encoding='utf-8') as f:
                json.dump(estadisticas_finales, f, indent=2, ensure_ascii=False)
            
            logger.info("📊 Estadísticas guardadas en riwi_carga_estadisticas.json")
            
        except Exception as e:
            logger.error(f"❌ Error guardando estadísticas: {e}")
    
    def ejecutar_carga_completa(self):
        """Ejecutar proceso completo de carga"""
        logger.info("=" * 60)
        logger.info("🚀 INICIANDO CARGA FINAL A POSTGRESQL")
        logger.info("=" * 60)
        
        # Validar CSV
        if not self.validar_csv():
            return False
        
        # Establecer conexiones
        if not self.conectar_postgresql():
            return False
        
        if not self.crear_engine_sqlalchemy():
            return False
        
        # Cargar CSV
        df = self.cargar_csv()
        if df is None:
            return False
        
        # Cargar dimensiones básicas
        if not self.cargar_dimensiones_basicas():
            return False
        
        # Procesar y cargar datos principales
        if not self.procesar_y_cargar(df):
            return False
        
        # Crear índices
        self.crear_indices_optimizacion()
        
        # Verificar carga
        self.verificar_carga()
        
        # Guardar estadísticas
        self.guardar_estadisticas()
        
        logger.info("=" * 60)
        logger.info("✅ CARGA FINAL COMPLETADA EXITOSAMENTE")
        logger.info("=" * 60)
        
        # Cerrar conexiones
        if self.conn:
            self.conn.close()
        
        return True

In [4]:
if __name__ == "__main__":
    # CONFIGURACIÓN - CAMBIA ESTOS VALORES
    ARCHIVO_CSV_LIMPIO = "riwi_datos_limpios.csv"  # ⚠️ CAMBIA SI ES NECESARIO
    
    # Crear cargador
    cargador = PostgreSQLLoaderFinal(DB_CONFIG, ARCHIVO_CSV_LIMPIO)
    
    # Ejecutar carga
    exito = cargador.ejecutar_carga_completa()
    
    if exito:
        print("🎉 ¡Carga a PostgreSQL completada!")
        print("📊 Revisa el log: riwi_loader.log")
        print("📈 Revisa las estadísticas: riwi_carga_estadisticas.json")
    else:
        print("❌ Carga fallida. Revisa el log.")

2026-01-05 17:17:26,869 - INFO - ============================================================
2026-01-05 17:17:26,870 - INFO - 🚀 INICIANDO CARGA FINAL A POSTGRESQL
2026-01-05 17:17:26,870 - INFO - ============================================================
2026-01-05 17:17:26,870 - INFO - 📊 Tamaño CSV: 272.27 MB
2026-01-05 17:17:26,875 - INFO - ✅ CSV válido encontrado: riwi_datos_limpios.csv
2026-01-05 17:17:26,875 - INFO - 📋 Columnas: ['fecha', 'producto', 'tipo_producto', 'cantidad', 'precio_unitario', 'ciudad', 'pais', 'tipo_venta', 'tipo_cliente', 'descuento', 'costo_envio', 'total', 'anio', 'mes', 'trimestre', 'nombre_mes', 'dia_semana', 'subtotal', 'monto_descuento', 'total_calculado']
2026-01-05 17:17:27,173 - INFO - 📈 Total filas: 1,935,733
2026-01-05 17:17:27,181 - INFO - ✅ Conexión psycopg2 establecida
2026-01-05 17:17:27,200 - INFO - ✅ Engine SQLAlchemy creado
2026-01-05 17:17:27,201 - INFO - 📖 Cargando CSV limpio...
2026-01-05 17:17:31,078 - INFO - ✅ CSV cargado: 1935733 r

🎉 ¡Carga a PostgreSQL completada!
📊 Revisa el log: riwi_loader.log
📈 Revisa las estadísticas: riwi_carga_estadisticas.json
